In [8]:
# Adrian Marinovich
# Springboard - Data Science Career Track 
# Capstone Project #2
# Feature extraction 

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from collections import deque
import csv
import cv2
import glob
import numpy as np
import operator
import os
import os.path
import pickle
import random
import shutil
import sys
import threading
import time
from tqdm import tqdm

from scipy.stats import reciprocal, uniform
from scipy.misc import imsave

import tensorflow as tf

# for reproducibility:
np.random.seed(41)
tf.reset_default_graph()
tf.set_random_seed(51)
random.seed(61)
os.environ['PYTHONHASHSEED'] = '0'
    
import keras
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.applications import VGG16
from keras import backend as K
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger
from keras.layers import Conv2D
from keras.layers import Input
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import ZeroPadding3D
from keras.layers.convolutional import (Conv2D, MaxPooling3D, Conv3D,
    MaxPooling2D)
from keras.layers.recurrent import LSTM
from keras.layers.recurrent import GRU
from keras.layers.wrappers import TimeDistributed
from keras.models import Model, load_model
from keras.models import model_from_yaml
from keras.models import Sequential
from keras.optimizers import SGD, Adam, RMSprop
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, load_img
from keras.utils import to_categorical

from sklearn.svm import LinearSVC 
from sklearn.svm import SVC 

from sklearn.datasets import fetch_mldata
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import SGDClassifier

# setup plots
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

Using TensorFlow backend.


In [ ]:
# Adapted from:
#   https://github.com/harvitronix/five-video-classification-methods
#   https://github.com/wushidonguc/two-stream-action-recognition-keras
#   https://github.com/fchollet/deep-learning-with-python-notebooks

In [3]:
# (adapted from:
#   https://github.com/harvitronix/five-video-classification-methods/blob/master/processor.py )

# Process image and return array

def process_image(image, target_shape):
    # Load the image.
    h, w, _ = target_shape
    image = load_img(image, target_size=(h, w))

    # Turn it into numpy, normalize and return.
    img_arr = img_to_array(image)
    x = (img_arr / 255.).astype(np.float32)

    return x

In [4]:
# (adapted from:
#    https://github.com/harvitronix/five-video-classification-methods/blob/master/data.py
#    https://github.com/wushidonguc/two-stream-action-recognition-keras/blob/master/spatial_train_data.py )

class threadsafe_iterator:
    def __init__(self, iterator):
        self.iterator = iterator
        self.lock = threading.Lock()

    def __iter__(self):
        return self

    def __next__(self):
        with self.lock:
            return next(self.iterator)

def threadsafe_generator(func):
    # Decorator
    def gen(*a, **kw):
        return threadsafe_iterator(func(*a, **kw))
    return gen

class DataSet():

    os.chdir('/home/adrian01/ucf101')
    
    def __init__(self, seq_length=0, class_limit=None, image_shape=(299, 299, 3)):
        # Constructor
        #   seq_length = (int) number of frames for fixed sequence length, or:
        #     0 ~> variable sequence length
        #   class_limit = (int) number of classes to limit the data to, or:
        #     None ~> no limit.

        # Set directory name where to put features once extracted
        #   Currently these are in place:
        #     features01 ~> features for 50-frame fixed sequences
        #     features02 ~> features for 80-frame fixed sequences
        #     features03 ~> features for 100-frame fixed sequences
        #     features04 ~> features for variable length sequences
        self.sequence_path = os.path.join('/home/adrian01/ucf101', 'features04')
        
        self.seq_length = seq_length
        self.class_limit = class_limit
        self.min_frames = 2  # min number of frames a video can have for us to use it
        self.max_frames = 2000  # max number of frames a video can have for us to use it

        # Get data
        self.data = self.get_data()

        # Get classes
        self.classes = self.get_classes()

        # Limit within min and max number of frames
        self.data = self.clean_data()
        self.image_shape = image_shape

    @staticmethod
    def get_data():
        # Load list of video metadata as a list 
        with open(os.path.join('/home/adrian01/data_file.csv'), 'r') as fin:
            reader = csv.reader(fin)
            data = list(reader)

        return data

    def clean_data(self):
        # Limit within min and max number of frames
        #  and to specified classes
        data_clean = []
        for item in self.data:
            if int(item[3]) >= self.min_frames and int(item[3]) <= self.max_frames \
                    and item[1] in self.classes:
                data_clean.append(item)

        return data_clean

    def get_classes(self):
        # Extract classes, and with specified limit
        classes = []
        for item in self.data:
            if item[1] not in classes:
                classes.append(item[1])

        # Sort
        classes = sorted(classes)

        # Return
        if self.class_limit is not None:
            return classes[:self.class_limit]
        else:
            return classes

    def get_class_one_hot(self, class_str):
        # Given class as string, return its number in classes list
        #  This lets us encode and one-hot it for training.
        
        # Encode it first
        label_encoded = self.classes.index(class_str)

        # Then one-hot it
        label_hot = to_categorical(label_encoded, len(self.classes))

        assert len(label_hot) == len(self.classes)

        return label_hot

    def split_train_test(self):
        # Split the data into train and test groups
        
        train = []
        test = []
        for item in self.data:
            if item[0] == 'train':
                train.append(item)
            else:
                test.append(item)
        return train, test

    def get_all_sequences_in_memory(self, train_test, data_type):
        # This is like our generator, but attempts to load everything 
        #   into memory, to train faster

        # Get the dataset
        train, test = self.split_train_test()
        data = train if train_test == 'train' else test

        print("Loading %d samples into memory for %sing." % (len(data), train_test))

        X, y = [], []
        for row in data:

            if data_type == 'images':
                frames = self.get_frames_for_sample(row)
                frames = self.rescale_list(frames, self.seq_length)

                # Build the image sequence
                sequence = self.build_image_sequence(frames)

            else:
                sequence = self.get_extracted_sequence(data_type, row)

                if sequence is None:
                    print("Can't find sequence. Did you generate them?")
                    raise

            X.append(sequence)
            y.append(self.get_class_one_hot(row[1]))

        return np.array(X), np.array(y)

    @threadsafe_generator
    def frame_generator(self, batch_size, train_test, data_type):
        # Makes a generator to train on,
        #  returning either data_type: 
        #    'features', 'images'

        # Get dataset for the generator
        train, test = self.split_train_test()
        data = train if train_test == 'train' else test

        print("Creating %s generator with %d samples." % (train_test, len(data)))

        while 1:
            X, y = [], []

            # Generate batch_size samples
            for _ in range(batch_size):
                # Reset to be safe
                sequence = None

                # Get a random sample
                sample = random.choice(data)

                # Check to see if we've already saved this sequence
                if data_type is "images":
                    # Get and resample frames
                    frames = self.get_frames_for_sample(sample)
                    frames = self.rescale_list(frames, self.seq_length)

                    # Build the image sequence
                    sequence = self.build_image_sequence(frames)
                else:
                    # Get the sequence from disk
                    sequence = self.get_extracted_sequence(data_type, sample)

                    if sequence is None:
                        raise ValueError("Can't find sequence. Did you generate them?")

                X.append(sequence)
                y.append(self.get_class_one_hot(sample[1]))

            yield np.array(X), np.array(y)

    def build_image_sequence(self, frames):
        # Given a set of frames (filenames), build sequence
        return [process_image(x, self.image_shape) for x in frames]

    def get_extracted_sequence(self, data_type, sample):
        # Get saved extracted features
        filename = sample[2]
        path = os.path.join(self.sequence_path, filename + '-' + str(self.seq_length) + \
            '-' + data_type + '.npy')
        if os.path.isfile(path):
            return np.load(path)
        else:
            return None

    def get_frames_by_filename(self, filename, data_type):
        # Given a filename for one of our samples, return the data
        
        # First, find the sample row.
        sample = None
        for row in self.data:
            if row[2] == filename:
                sample = row
                break
        if sample is None:
            raise ValueError("Couldn't find sample: %s" % filename)

        if data_type == "images":
            # Get and resample frames.
            frames = self.get_frames_for_sample(sample)
            frames = self.rescale_list(frames, self.seq_length)
            # Build the image sequence
            sequence = self.build_image_sequence(frames)
        else:
            # Get the sequence from disk.
            sequence = self.get_extracted_sequence(data_type, sample)

            if sequence is None:
                raise ValueError("Can't find sequence. Did you generate them?")

        return sequence

    @staticmethod
    def get_frames_for_sample(sample):
        # Given a video row from video metadata file, get corresponding frame filenames
        path = os.path.join('/home/adrian01/ucf101', sample[0], sample[1])
        filename = sample[2]
        images = sorted(glob.glob(os.path.join(path, filename + '*jpg')))
        return images

    @staticmethod
    def get_filename_from_image(filename):
        parts = filename.split(os.path.sep)
        return parts[-1].replace('.jpg', '')

    @staticmethod
    def rescale_list(input_list, size):
        # Return list of frames at specified fixed sequence length, or:
        #   Note that size==0 will cause output of entire image sequence
        
        if size == 0:
            outpt = [input_list[i] for i in range(0, len(input_list))]
            return outpt
        
        elif len(input_list) > size:
            skip = len(input_list) // size
            outpt = [input_list[i] for i in range(0, len(input_list), skip)]
            return outpt[:size]

        elif len(input_list) == size:
            outpt = [input_list[i] for i in range(0, len(input_list))]
            return outpt[:size]

        elif len(input_list) < size and len(input_list) > 2:
            add = 1 - (len(input_list) / size)
            a_c = 0
            x_c = 0
            outpt = []
            list_end = len(input_list) - 1
            for x in range(size):
                a_c += add
                x_c = np.ceil(x - a_c)
                x_c = x_c.astype(np.int16)
                if x_c <= list_end:
                    outpt.append(input_list[x_c])
                elif x_c > list_end:
                    outpt.append(input_list[list_end])
            return outpt[:size]

        else:
            outpt = []
            for x in range(size):
                outpt.append(input_list[0])
            return outpt[:size]
    
    def print_class_from_prediction(self, predictions, nb_to_return=5):
        # Given a prediction, print the top classes

        # Get the prediction for each label
        label_predictions = {}
        for i, label in enumerate(self.classes):
            label_predictions[label] = predictions[i]

        # Sort
        sorted_lps = sorted(
            label_predictions.items(),
            key=operator.itemgetter(1),
            reverse=True
        )

        # And return the top N
        for i, class_prediction in enumerate(sorted_lps):
            if i > nb_to_return - 1 or class_prediction[1] == 0.0:
                break
            print("%s: %.2f" % (class_prediction[0], class_prediction[1]))

In [13]:
# (adapted from:
#    https://github.com/harvitronix/five-video-classification-methods/blob/master/extractor.py )

class Extractor():
    
    os.chdir('/home/adrian01/ucf101')
    
    def __init__(self, weights=None):
        # Either load pretrained from imagenet, or 
        #   load saved weights from our own training

        self.weights = weights  # so we can check elsewhere which model

        if weights is None:
            # Get model with pretrained weights
            base_model = InceptionV3(
                weights='imagenet',
                include_top=True
            )
            
            # Extract features at the final pool layer.
            self.model = Model(
                inputs=base_model.input,
                outputs=base_model.get_layer('avg_pool').output
            )            
            
        else:
            # Load the model first.
            self.model = load_model(weights)

            # Then remove the top so we get features, not predictions
            # From: https://github.com/fchollet/keras/issues/2371
            self.model.layers.pop()
            self.model.layers.pop()  # two pops to get to pool layer
            self.model.outputs = [self.model.layers[-1].output]
            self.model.output_layers = [self.model.layers[-1]]
            self.model.layers[-1].outbound_nodes = []

    def extract(self, image_path):
        img = load_img(image_path, target_size=(299, 299))
        x = img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)

        # Get the prediction
        features = self.model.predict(x)

        if self.weights is None:
            # For imagenet/default network:
            features = features[0]
        else:
            # For loaded network:
            features = features[0]

        return features

In [14]:
# DONE - DO NOT RUN
# COMPLETED WITH 13320 ITERATIONS - 50-frame length in /features01
#                                   80-frame length in /features02
#                                   100-frame length in /features03
#                                   variable-length features in /features04
#
# (adapted from:
#    https://github.com/harvitronix/five-video-classification-methods/blob/master/extract_features.py )

# Generates extracted features for each video, bundled into array 
#   with (sequence length, 2048) dimension


# Class_limit is an integer that denotes the first N classes you want to extract features from

# Set defaults.

os.chdir('/home/adrian01/ucf101')

seq_length = 0
class_limit = None  # Number of classes to extract. Can be 1-101 or None for all.

# Get the dataset.
data = DataSet(seq_length=seq_length, class_limit=class_limit)

# get the model.
model = Extractor()

# Loop through data.
pbar = tqdm(total=len(data.data))
for video in data.data:

    # Get the path to the sequence for this video.
    path = os.path.join('/home/adrian01/ucf101', 'features04', video[2] + '-' + str(video[3]) + \
        '-features')  # numpy will auto-append .npy

    # Check if we already have it.
    if os.path.isfile(path + '.npy'):
        pbar.update(1)
        continue

    # Get the frames for this video.
    frames = data.get_frames_for_sample(video)

    # Now downsample to just the ones we need.
    frames = data.rescale_list(frames, seq_length)

    # Now loop through and extract features to build the sequence.
    sequence = []
    for image in frames:
        features = model.extract(image)
        sequence.append(features)

    # Save the sequence.
    np.save(path, sequence)

    pbar.update(1)

pbar.close()

100%|██████████| 13320/13320 [17:15:33<00:00,  4.55s/it]   


In [16]:
chek = np.load('/home/adrian01/ucf101/features04/v_Swing_g08_c01-0-features.npy')
chek.shape

#should be (160, 2048)

(160, 2048)

In [15]:
# RUN WHEN DONE:
#K.clear_session()